# <font color='cornflowerblue'>Natural Language Inference Approach UM - Data Team Club

In the second UM Data Team Club project, we'll be diving into Natural Language Inference (NLI). Our goal is to explore and analyze the relationships between different text pairs to understand whether one sentence logically follows from another.

## Contents Table
 - [Imports](#1)
 - [Accelerator](#2)
 - [Load Data](#3)
 - [Data Exploring](#4)
 - [Data Preprocessing](#5)
 - [BERT Base Multi Model](#6)

In [91]:
# !pip install keras_nlp
# !pip install seaborn
# !pip install wordcloud

### Imports <a id='1'></a>

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import os

### Accelerator <a id='2'></a>

### Load Data <a id='3'></a>

In [93]:
df_train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
print(f"Train dataset size: {df_train.shape}")

Train dataset size: (12120, 6)


### Data Exploring <a id='4'></a>

In [94]:
df_train.head(3)

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0


The NLI model will assign labels of 0, 1, or 2 (corresponding to entailment, neutral, and contradiction) to pairs of premises and hypotheses.

| <span style="color:cornflowerblue">Label</span> | <span style="color:cornflowerblue">Description</span>       |
|---------------------------------------|---------------------------------------------------|
| 0                                     | Entailment                                        |
| 1                                     | Neutral                                           |
| 2                                     | Contradiction                                     |


### Generative AI classification <a id='6'></a>

In [95]:
#!pip install langchain
#!pip install langchain_community

In [96]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# Pass the directory path where the model is stored on your system
# -- model_name = "google/flan-t5-large"

# Pass the namespace/repo_name to download the repo to your machine
model_name = "google/flan-t5-large"

# Initialize a tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize a model for sequence-to-sequence tasks using the specified pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [97]:
# Create a pipeline for text-to-text generation using a specified model and tokenizer
pipe = pipeline(
    "text2text-generation",  # Specify the task as text-to-text generation
    model=model,             # Use the previously initialized model
    tokenizer=tokenizer,     # Use the previously initialized tokenizer
    max_length=5,            # Set the maximum length for generated text to 512 tokens
    temperature=0,           # Set the temperature parameter for controlling randomness (0 means deterministic)
    top_p=0.95,              # Set the top_p parameter for controlling the nucleus sampling (higher values make output more focused)
    repetition_penalty=1.15, # Set the repetition_penalty to control the likelihood of repeated words or phrases
)

# Create a Hugging Face pipeline for local language model (LLM) using the 'pipe' pipeline
local_llm = HuggingFacePipeline(pipeline=pipe)

In [98]:
df_train[100:110]

,id,premise,hypothesis,lang_abv,language,label
100,7cc9bb38f9,"The Women's Haven, which provides shelter and ...",There is a full-time attorney at the Women's H...,en,English,0
101,a8bcd9ccf7,uh high humidity,Air with increased water content.,en,English,0
102,6c2527b960,"Trong trường hợp của Mỹ 11, thông tin liên lạc...",Không có bất cứ thông tin liên lạc gì từ Ameri...,vi,Vietnamese,2
103,80f6ac92a4,We need to look at the implications that these...,We can completely ignore the implications of t...,en,English,2
104,eb04f1a8f0,yeah yeah yeah well because that's the way the...,There is another layer to the surface.,en,English,0
105,e7ad6c38fc,"In this situation, the value to the mailer of ...",No consideration will be given to the cost of ...,en,English,2
106,ca2f6459bd,Is there adequate information for judging gene...,Output information is way too much generalized.,en,English,1
107,212fc2ad66,The political cleansing that did not happen th...,The impeachment process involved Clinton's hus...,en,English,1
108,df8998b7a7,嗯，我的祖父母总是非常非常的有爱心，而我的父母也是，我们在下边享受美好时光。,我的祖父母是一对非常恩爱的夫妻。,zh,Chinese,0
109,d5249d1219,isn't it i can remember i've only been here ei...,I have only been here for two years.,en,English,2


In [99]:
# Ejemplo con 1 sola clasificacion 
premise = df_train["premise"][100]
hypothesis = df_train["hypothesis"][100]

print(local_llm(f'''
You are given a premise and a hypothesis, classify the hypothesis based on the premise into: Entailment, Neutral or Contradiction.
The premise is {premise}, and the hypothesis to classify is {hypothesis}'''))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Entailment


In [100]:
k = 150
clasiff = []
for i in range(len(df_train.head(k))):
    premise = df_train["premise"][i]
    hypothesis = df_train["hypothesis"][i]
    
    pred = local_llm(f'''
    You are given a premise and a hypothesis, classify the hypothesis based on the premise into: Entailment, Neutral or Contradiction.
    The premise is {premise}, and the hypothesis to classify is {hypothesis}''')
    
    clasiff += [pred]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606:

In [107]:
clasiff[:15]

[0, 2, 0, 0, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1]

In [102]:
for i in range(len(clasiff)):
    if clasiff[i] == "Entailment":
        clasiff[i] = 0
    elif clasiff[i] == "Neutral":
        clasiff[i] = 1
    else:
        clasiff[i] = 2

In [106]:
clasiff[:15]

[0, 2, 0, 0, 1]

In [104]:
from sklearn.metrics import accuracy_score
score = accuracy_score(df_train.label.head(k), clasiff)
print(score)

0.6266666666666667
